In [33]:
import os
import re
import ast
import json
import torch
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib as mpl

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from tqdm import tqdm
from umap import UMAP
from torch.nn import CosineSimilarity
from scipy.stats import pointbiserialr
from datasets import load_dataset, load_from_disk
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel

tqdm.pandas()

# Seaborn settings
sns.set_context("notebook")
sns.set_palette("colorblind")
sns.color_palette("pastel")
aug_regex = re.compile(r"<aug>(.*?)</aug>", re.DOTALL)

# Matplotlib settings
# os.environ["PATH"] += os.pathsep + '/Library/TeX/texbin'
# mpl.rcParams.update(mpl.rcParamsDefault)
# plt.rcParams.update({
#   "text.usetex": True,
#   "font.family": "serif",
#   "font.serif": "cm",
#   "font.size": 16,
# })


## Abstract & Intro

In [34]:
# sentiment_bert_delta = 56.91 - 52.05
# print(f"Sentiment BERT delta: {sentiment_bert_delta:.2f}")
# toxic_bert_delta = 60.75 - 53.90
# # toxic_bert_delta = 64.36 - 53.90
# print(f"Toxicity BERT delta: {toxic_bert_delta:.2f}")
# news_bert_delta = 89.75 - 88.57
# print(f"News BERT delta: {news_bert_delta:.2f}")

# mean_bert_delta = (sentiment_bert_delta + toxic_bert_delta + news_bert_delta) / 3
# print(f"Mean BERT delta: {mean_bert_delta:.2f}\n")

In [35]:
# sentiment_t5_delta = 59.83 - 57.99
# print(f"Sentiment T5 delta: {sentiment_t5_delta:.2f}")
# toxic_t5_delta = 63.89 - 58.81
# print(f"Toxicity T5 delta: {toxic_t5_delta:.2f}")
# news_t5_delta = 90.53 - 89.01
# print(f"News T5 delta: {news_t5_delta:.2f}")

# mean_t5_delta = (sentiment_t5_delta + toxic_t5_delta + news_t5_delta) / 3
# print(f"Mean T5 delta: {mean_t5_delta:.2f}\n")

In [36]:
# sentiment_falcon_delta = 44.20 - 47.16
# print(f"Sentiment Falcon delta: {sentiment_falcon_delta:.2f}")
# toxic_falcon_delta = 58.41 - 66.68
# print(f"Toxicity Falcon delta: {toxic_falcon_delta:.2f}")
# news_falcon_delta = 28.16 - 25.96
# print(f"News Falcon delta: {news_falcon_delta:.2f}")

# mean_falcon_delta = (sentiment_falcon_delta + toxic_falcon_delta + news_falcon_delta) / 3
# print(f"Mean Falcon delta: {mean_falcon_delta:.2f}\n")

In [37]:
# sentiment_augmentation_rate = 56.51
# print(f"Sentiment augmentation rate: {sentiment_augmentation_rate:.2f}")
# toxic_augmentation_rate = 66.46
# print(f"Toxicity augmentation rate: {toxic_augmentation_rate:.2f}")
# news_augmentation_rate = 3.76
# print(f"News augmentation rate: {news_augmentation_rate:.2f}")
# mean_augmentation_rate = (sentiment_augmentation_rate + toxic_augmentation_rate + news_augmentation_rate) / 3
# reduction = 100 - mean_augmentation_rate
# print(f"Mean augmentation rate: {reduction:.2f}")

In [38]:
# sentiment_translate_bert_delta = 52.60 - 52.05
# print(f"Sentiment BERT delta: {sentiment_translate_bert_delta:.2f}")
# toxic_translate_bert_delta = 61.65 - 53.90
# print(f"Toxicity BERT delta: {toxic_translate_bert_delta:.2f}")
# news_translate_bert_delta = 88.82 - 88.57
# print(f"News BERT delta: {news_translate_bert_delta:.2f}")

# mean_translate_bert_delta = (sentiment_translate_bert_delta + toxic_translate_bert_delta + news_translate_bert_delta) / 3
# print(f"Mean BERT delta: {mean_translate_bert_delta:.2f}\n")

In [39]:
# sentiment_insert_bert_delta = 51.92 - 52.05
# print(f"Sentiment BERT delta: {sentiment_insert_bert_delta:.2f}")
# toxic_insert_bert_delta = 53.10 - 53.90
# print(f"Toxicity BERT delta: {toxic_insert_bert_delta:.2f}")
# news_insert_bert_delta = 88.84 - 88.57
# print(f"News BERT delta: {news_insert_bert_delta:.2f}")

# mean_insert_bert_delta = (sentiment_insert_bert_delta + toxic_insert_bert_delta + news_insert_bert_delta) / 3
# print(f"Mean BERT delta: {mean_insert_bert_delta:.2f}\n")

In [40]:
# sentiment_sub_bert_delta = 51.78 - 52.05
# print(f"Sentiment BERT delta: {sentiment_sub_bert_delta:.2f}")
# toxic_sub_bert_delta = 53.37 - 53.90
# print(f"Toxicity BERT delta: {toxic_sub_bert_delta:.2f}")
# news_sub_bert_delta = 88.86 - 88.57
# print(f"News BERT delta: {news_sub_bert_delta:.2f}")

# mean_sub_bert_delta = (sentiment_sub_bert_delta + toxic_sub_bert_delta + news_sub_bert_delta) / 3
# print(f"Mean BERT delta: {mean_sub_bert_delta:.2f}\n")

In [41]:
# sentiment_id_bert_delta = 90.85 - 90.38
# print(f"Sentiment BERT delta: {sentiment_id_bert_delta:.2f}")
# toxic_id_bert_delta = 91.45 - 88.46
# print(f"Toxicity BERT delta: {toxic_id_bert_delta:.2f}")
# news_id_bert_delta = 93.53 - 94.43
# print(f"News BERT delta: {news_id_bert_delta:.2f}")

# mean_id_bert_delta = (sentiment_id_bert_delta + toxic_id_bert_delta + news_id_bert_delta) / 3
# print(f"Mean BERT delta: {mean_id_bert_delta:.2f}\n")

In [42]:
# set seeds
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [43]:
# display full dataframes
pd.set_option("display.max_columns", None)

In [44]:
# plotting constants
TITLE_FONT_SIZE = 16
WSPACE = 0.3
FIGURE_HEIGHT = 3
LINE_WIDTH = 2
FIG_SIZE = 4
MARKER_SIZE = 8
X_LABEL_ROTATION = 20
TTA_METHOD_ORDER = {
    "Insert": 0,
    "Substitute": 1,
    "Translate": 2,
    "LLM-TTA: Paraphrase": 3,
    "LLM-TTA: ICR": 4,
}

In [45]:

inference_logs = load_from_disk("data/combined_dataset_multiseed")
list(inference_logs.keys())

['seed=3_BOSS_Sentiment_SST5_BERT_Insert',
 'seed=3_BOSS_Sentiment_SST5_BERT_Substitute',
 'seed=3_BOSS_Sentiment_SST5_BERT_Translate',
 'seed=3_BOSS_Sentiment_SST5_BERT_Paraphrase',
 'seed=3_BOSS_Sentiment_SST5_BERT_ICR',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Substitute',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Translate',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Paraphrase',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT1500_ICR',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Insert',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Substitute',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Translate',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Paraphrase',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT3000_ICR',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Insert',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Substitute',
 'seed=3_BOSS_Sentiment_Ablate_Data_SST5_BERT6000

In [46]:
"_".join("seed=58_BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Paraphrase".split("_")[1:])

'BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Paraphrase'

In [47]:
def parse_task_name(split_name):
    return "Sentiment" if "Sentiment" in split_name else "Toxicity" if "Toxicity" in split_name else "News"

def parse_distribution(split_name):
    return split_name.split("_")[-3]

def parse_model(split_name):
    return split_name.split("_")[-2]

def parse_tta_method(split_name):
    return split_name.split("_")[-1]

def parse_seed(split_name):
    return int(split_name.split("_")[0].split("seed=")[-1])

def format_split_name(split_name):
    return "_".join(split_name.split("_")[1:])


formatted_method_names = {
    "Insert": "Insert",
    "Substitute": "Substitute",
    "Translate": "Translate",
    "Paraphrase": "LLM-TTA: Paraphrase",
    "ICR": "LLM-TTA: ICR",
}
    

In [48]:
multiseed_inference_logs = {}
for split_name in tqdm(inference_logs):
    seed = parse_seed(split_name)
    if seed not in multiseed_inference_logs:
        multiseed_inference_logs[seed] = {}
    
    formatted_split_name = format_split_name(split_name)
    multiseed_inference_logs[seed][formatted_split_name] = inference_logs[split_name]

multiseed_inference_logs

100%|██████████| 1120/1120 [00:00<00:00, 249289.99it/s]

{3: {'BOSS_Sentiment_SST5_BERT_Insert': Dataset({
      features: ['outcome', 'original_text', 'augmentations', 'generations', 'original_predicted_class', 'tta_predicted_class', 'label', 'tta_inference_latency', 'original_prediction_entropy', 'tta_prediction_entropy', 'prediction_entropy_decreased', 'prediction_entropy_decrease', 'tta_mean_class_probs', 'tta_all_class_probs', 'tta_all_class_entropy'],
      num_rows: 1068
  }),
  'BOSS_Sentiment_SST5_BERT_Substitute': Dataset({
      features: ['outcome', 'original_text', 'augmentations', 'generations', 'original_predicted_class', 'tta_predicted_class', 'label', 'tta_inference_latency', 'original_prediction_entropy', 'tta_prediction_entropy', 'prediction_entropy_decreased', 'prediction_entropy_decrease', 'tta_mean_class_probs', 'tta_all_class_probs', 'tta_all_class_entropy'],
      num_rows: 1068
  }),
  'BOSS_Sentiment_SST5_BERT_Translate': Dataset({
      features: ['outcome', 'original_text', 'augmentations', 'generations', 'origina

## No TTA Baselines

In [49]:
no_tta_accuracies = {}
for seed in multiseed_inference_logs:
    no_tta_accuracies[seed] = {}
    for split_name in tqdm(multiseed_inference_logs[seed], desc=f"Seed = {seed}"):
        split_frame = multiseed_inference_logs[seed][split_name].to_pandas()
        split_no_tta_accuracy = classification_report(split_frame["label"], split_frame["original_predicted_class"], output_dict=True, zero_division=0.0)["accuracy"]
        no_tta_accuracies[seed][split_name] = split_no_tta_accuracy

no_tta_accuracies

Seed = 58: 100%|██████████| 280/280 [00:51<00:00,  5.40it/s]


{3: {'BOSS_Sentiment_SST5_BERT_Insert': 0.6844569288389513,
  'BOSS_Sentiment_SST5_BERT_Substitute': 0.6844569288389513,
  'BOSS_Sentiment_SST5_BERT_Translate': 0.6850467289719626,
  'BOSS_Sentiment_SST5_BERT_Paraphrase': 0.6844569288389513,
  'BOSS_Sentiment_SST5_BERT_ICR': 0.6844569288389513,
  'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert': 0.6825842696629213,
  'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Substitute': 0.6825842696629213,
  'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Translate': 0.6828358208955224,
  'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Paraphrase': 0.6825842696629213,
  'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_ICR': 0.6825842696629213,
  'BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Insert': 0.6779026217228464,
  'BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Substitute': 0.6779026217228464,
  'BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Translate': 0.6791044776119403,
  'BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Paraphrase': 0.6779026217228464,
  'BOSS_Sentiment_Ablate_Dat

## Analyze Main Results

In [50]:
multiseed_split_data = {}
for seed in multiseed_inference_logs:
    main_results_bert_splits = [split for split in multiseed_inference_logs[seed].keys() if "Ablate" not in split]
    datasets = ["BOSS_Sentiment", "BOSS_Toxicity", "AgNewsTweets"]
    split_data = {}

    for task_name in datasets:
        if task_name not in split_data:
            split_data[task_name] = {}

        for split in tqdm(main_results_bert_splits, desc=f"Processing splits for {task_name} Seed={seed}"):
            if task_name in split:
                distribution = split.split("_")[-3]
                model = split.split("_")[-2]
                tta_method = split.split("_")[-1]
                baseline_accuracy = classification_report(multiseed_inference_logs[seed][split]["label"], multiseed_inference_logs[seed][split]["original_predicted_class"], output_dict=True, zero_division=0.0)["accuracy"]
                tta_accuracy = classification_report(multiseed_inference_logs[seed][split]["label"], multiseed_inference_logs[seed][split]["tta_predicted_class"], output_dict=True, zero_division=0.0)["accuracy"]
                accuracy_delta = tta_accuracy - baseline_accuracy

                if model not in split_data[task_name]:
                    split_data[task_name][model] = {}

                if tta_method not in split_data[task_name][model]:
                    split_data[task_name][model][tta_method] = {}

                split_data[task_name][model][tta_method][distribution] = {
                    "distribution": distribution,
                    "model": model,
                    "tta_method": tta_method,
                    "baseline_accuracy": baseline_accuracy,
                    "tta_accuracy": tta_accuracy,
                    "accuracy_delta": accuracy_delta
                }

        # get the Accuracy Gain for each method excluding ID
        for model_name in split_data[task_name]:
            for tta_method in split_data[task_name][model_name]:
                accuracies = []
                accuracy_deltas = []
                baseline_accuracies = []
                for distribution in split_data[task_name][model_name][tta_method]:
                    if distribution == "ID":
                        continue

                    current_tta_result = split_data[task_name][model_name][tta_method][distribution]
                    accuracies.append(current_tta_result["tta_accuracy"])
                    accuracy_deltas.append(current_tta_result["accuracy_delta"])
                    baseline_accuracies.append(current_tta_result["baseline_accuracy"])

                split_data[task_name][model_name][tta_method]["mean_accuracy"] = np.mean(accuracies)
                split_data[task_name][model_name][tta_method]["mean_accuracy_delta"] = np.mean(accuracy_deltas)
                split_data[task_name][model_name][tta_method]["mean_baseline_accuracy"] = np.mean(baseline_accuracies)

    # print(json.dumps(split_data, indent=4))
    multiseed_split_data[seed] = split_data

print(json.dumps(multiseed_split_data, indent=4))

Processing splits for BOSS_Toxicity Seed=17:  49%|████▊     | 51/105 [00:00<00:00, 432.61it/s]

Processing splits for AgNewsTweets Seed=58: 100%|██████████| 105/105 [00:00<00:00, 135.01it/s]

{
    "3": {
        "BOSS_Sentiment": {
            "BERT": {
                "Insert": {
                    "SST5": {
                        "distribution": "SST5",
                        "model": "BERT",
                        "tta_method": "Insert",
                        "baseline_accuracy": 0.6844569288389513,
                        "tta_accuracy": 0.6619850187265918,
                        "accuracy_delta": -0.022471910112359494
                    },
                    "SemEval": {
                        "distribution": "SemEval",
                        "model": "BERT",
                        "tta_method": "Insert",
                        "baseline_accuracy": 0.4497623896809233,
                        "tta_accuracy": 0.44937445446610413,
                        "accuracy_delta": -0.00038793521481916837
                    },
                    "Dynasent": {
                        "distribution": "Dynasent",
                        "model": "BERT",
               

In [51]:
multiseed_detailed_results = {}
for seed in multiseed_split_data:
    multiseed_detailed_results[seed] = {}
    detailed_results_records = []
    for task_name in multiseed_split_data[seed]:
        for model_name in multiseed_split_data[seed][task_name]:
            for tta_method in multiseed_split_data[seed][task_name][model_name]:
                for dist in [dist for dist in multiseed_split_data[seed][task_name][model_name][tta_method] if not dist.startswith("mean")]:
                    perf_record = multiseed_split_data[seed][task_name][model_name][tta_method][dist]
                    detailed_results_records.append({
                        "task": task_name,
                        "model": model_name,
                        "tta_method": tta_method,
                        "distribution": dist,
                        "mean_accuracy": perf_record["tta_accuracy"] * 100,
                    })
                    detailed_results_records.append({
                        "task": task_name,
                        "model": model_name,
                        "tta_method": "None",
                        "distribution": dist,
                        "mean_accuracy": perf_record["baseline_accuracy"] * 100,
                    })

    print(f"Seed: {seed}")

    detailed_results_frame = pd.DataFrame(detailed_results_records)
    for task_name in multiseed_split_data[seed]:
        task_frame = detailed_results_frame[detailed_results_frame["task"] == task_name]
        display(task_frame.groupby(["task", "model", "distribution", "tta_method", ]).mean().round(2).T)

    full_sweep_results = detailed_results_frame.groupby(["task", "model", "distribution", "tta_method", ]).mean().round(2)
    full_sweep_results.to_csv(f"data/seed={seed}_detailed_results.csv")
    multiseed_detailed_results[seed] = full_sweep_results

print("Across Seeds")
multiseed_detailed_results

Seed: 3


task          BOSS_Sentiment                                                \
model                   BERT                                                 
distribution        Dynasent                                                 
tta_method               ICR Insert   None Paraphrase Substitute Translate   
mean_accuracy          48.56  41.74  42.71      46.48      42.62      43.4   

task                                                                       \
model                                                                       
distribution   SST5                                               SemEval   
tta_method      ICR Insert   None Paraphrase Substitute Translate     ICR   
mean_accuracy  73.6   66.2  68.46       72.1      67.79     68.32   49.02   

task                                                                         \
model                                                         Falcon          
distribution                                                Dynasent          
tta_method    Insert   None Paraphrase Substitute Translate      ICR Insert   
mean_accuracy  44.94  44.98      47.95      44.84     45.45    40.42  41.48   

task                                                                       \
model                                                                       
distribution                                           SST5                 
tta_method      None Paraphrase Substitute Translate    ICR Insert   None   
mean_accuracy  42.87      41.57      41.62     41.23  51.03  54.03  56.74   

task                                                                 \
model                                                                 
distribution                                  SemEval                 
tta_method    Paraphrase Substitute Translate     ICR Insert   None   
mean_accuracy      53.93      52.15     51.68   39.74  40.77  40.69   

task                                                                  \
model                                               T5                 
distribution                                  Dynasent                 
tta_method    Paraphrase Substitute Translate      ICR Insert   None   
mean_accuracy      40.45      38.11     39.26    52.27   45.6  47.75   

task                                                                           \
model                                                                           
distribution                                    SST5                            
tta_method    Paraphrase Substitute Translate    ICR Insert   None Paraphrase   
mean_accuracy       50.3       44.1     47.11  75.75  72.28  76.23      77.06   

task                                                                 \
model                                                                 
distribution                       SemEval                            
tta_method    Substitute Translate     ICR Insert   None Paraphrase   
mean_accuracy      70.51     73.74   51.11  49.67  50.28      50.86   

task                                
model                               
distribution                        
tta_method    Substitute Translate  
mean_accuracy      48.86     49.29

task          BOSS_Toxicity                                                \
model                  BERT                                                 
distribution       AdvCivil                                                 
tta_method              ICR Insert   None Paraphrase Substitute Translate   
mean_accuracy         51.45  26.09  30.43      63.29      27.54     55.56   

task                                                                      \
model                                                                      
distribution  ImplicitHate                                                 
tta_method             ICR Insert   None Paraphrase Substitute Translate   
mean_accuracy        65.64  65.03  64.53      64.96      66.05     64.34   

task                                                                          \
model                                                                 Falcon   
distribution  Toxigen                                               AdvCivil   
tta_method        ICR Insert   None Paraphrase Substitute Translate      ICR   
mean_accuracy   66.14  66.99  66.67      66.14      65.92     66.35     77.9   

task                                                                      \
model                                                                      
distribution                                                ImplicitHate   
tta_method    Insert   None Paraphrase Substitute Translate          ICR   
mean_accuracy  81.64  81.64      79.95      81.64     81.64        44.45   

task                                                                        \
model                                                                        
distribution                                                Toxigen          
tta_method    Insert   None Paraphrase Substitute Translate     ICR Insert   
mean_accuracy  41.68  43.26      43.53      44.46     43.01   52.44   48.2   

task                                                                         \
model                                                      T5                 
distribution                                         AdvCivil                 
tta_method      None Paraphrase Substitute Translate      ICR Insert   None   
mean_accuracy  52.44      52.97      50.11     53.08     59.9  42.03  46.86   

task                                                                      \
model                                                                      
distribution                                  ImplicitHate                 
tta_method    Paraphrase Substitute Translate          ICR Insert   None   
mean_accuracy      71.62      40.94      59.3        64.93  64.62  63.93   

task                                                                 \
model                                                                 
distribution                                  Toxigen                 
tta_method    Paraphrase Substitute Translate     ICR Insert   None   
mean_accuracy      64.42      65.34     63.92   66.24  65.71  65.71   

task                                           
model                                          
distribution                                   
tta_method    Paraphrase Substitute Translate  
mean_accuracy      66.35      62.85     66.99

task          AgNewsTweets                                                \
model                 BERT                                                 
distribution        Tweets                                                 
tta_method             ICR Insert   None Paraphrase Substitute Translate   
mean_accuracy        89.66  88.94  88.57      89.03      88.67      88.5   

task                                                                       \
model         Falcon                                                   T5   
distribution  Tweets                                               Tweets   
tta_method       ICR Insert   None Paraphrase Substitute Translate    ICR   
mean_accuracy   27.6  25.11  25.86      27.65      24.77     26.05  90.23   

task                                                         
model                                                        
distribution                                                 
tta_method    Insert   None Paraphrase Substitute Translate  
mean_accuracy   89.7  88.99      89.84      89.09      89.0

Seed: 17


...

task          BOSS_Sentiment                                                \
model                   BERT                                                 
distribution        Dynasent                                                 
tta_method               ICR Insert   None Paraphrase Substitute Translate   
mean_accuracy          49.05  42.04  42.71      45.74      42.52      43.4   

task                                                                        \
model                                                                        
distribution    SST5                                               SemEval   
tta_method       ICR Insert   None Paraphrase Substitute Translate     ICR   
mean_accuracy  73.78  68.35  68.46      71.54       67.7     68.41   49.24   

task                                                                         \
model                                                         Falcon          
distribution                                                Dynasent          
tta_method    Insert   None Paraphrase Substitute Translate      ICR Insert   
mean_accuracy  44.95  44.98      48.11      44.81     45.45    40.58  41.88   

task                                                                       \
model                                                                       
distribution                                           SST5                 
tta_method      None Paraphrase Substitute Translate    ICR Insert   None   
mean_accuracy  42.87      41.37      41.55     41.23  50.94  55.24  56.74   

task                                                                 \
model                                                                 
distribution                                  SemEval                 
tta_method    Paraphrase Substitute Translate     ICR Insert   None   
mean_accuracy      53.75      50.75     51.68   39.52  40.89  40.69   

task                                                                  \
model                                               T5                 
distribution                                  Dynasent                 
tta_method    Paraphrase Substitute Translate      ICR Insert   None   
mean_accuracy      40.72      37.76     39.26    52.78  46.62  47.75   

task                                                                           \
model                                                                           
distribution                                    SST5                            
tta_method    Paraphrase Substitute Translate    ICR Insert   None Paraphrase   
mean_accuracy      49.98      45.37     47.11  75.66  72.19  76.23      76.12   

task                                                                 \
model                                                                 
distribution                       SemEval                            
tta_method    Substitute Translate     ICR Insert   None Paraphrase   
mean_accuracy      69.57     73.74   50.97  49.74  50.28      50.53   

task                                
model                               
distribution                        
tta_method    Substitute Translate  
mean_accuracy      49.07     49.29

task          BOSS_Toxicity                                                \
model                  BERT                                                 
distribution       AdvCivil                                                 
tta_method              ICR Insert   None Paraphrase Substitute Translate   
mean_accuracy         52.42  26.93  30.43      62.08      26.33     55.56   

task                                                                      \
model                                                                      
distribution  ImplicitHate                                                 
tta_method             ICR Insert   None Paraphrase Substitute Translate   
mean_accuracy        65.65  64.96  64.53      64.93      66.13     64.34   

task                                                                          \
model                                                                 Falcon   
distribution  Toxigen                                               AdvCivil   
tta_method        ICR Insert   None Paraphrase Substitute Translate      ICR   
mean_accuracy   65.82  66.99  66.67      66.03       65.5     66.35    78.14   

task                                                                      \
model                                                                      
distribution                                                ImplicitHate   
tta_method    Insert   None Paraphrase Substitute Translate          ICR   
mean_accuracy  81.64  81.64      80.07      81.64     81.64        44.23   

task                                                                        \
model                                                                        
distribution                                                Toxigen          
tta_method    Insert   None Paraphrase Substitute Translate     ICR Insert   
mean_accuracy   41.5  43.25      43.64      44.48     43.02   52.02  49.15   

task                                                                         \
model                                                      T5                 
distribution                                         AdvCivil                 
tta_method      None Paraphrase Substitute Translate      ICR Insert   None   
mean_accuracy  52.44      52.87      50.42     53.08    59.78  41.67  46.86   

task                                                                      \
model                                                                      
distribution                                  ImplicitHate                 
tta_method    Paraphrase Substitute Translate          ICR Insert   None   
mean_accuracy      69.08      42.63      59.3        64.81  64.64  63.93   

task                                                                 \
model                                                                 
distribution                                  Toxigen                 
tta_method    Paraphrase Substitute Translate     ICR Insert   None   
mean_accuracy      64.35      65.02     63.92    67.2  65.92  65.71   

task                                           
model                                          
distribution                                   
tta_method    Paraphrase Substitute Translate  
mean_accuracy      66.24      63.59     66.99

task          AgNewsTweets                                                \
model                 BERT                                                 
distribution        Tweets                                                 
tta_method             ICR Insert   None Paraphrase Substitute Translate   
mean_accuracy         89.7  88.91  88.57      88.94      88.73      88.5   

task                                                                       \
model         Falcon                                                   T5   
distribution  Tweets                                               Tweets   
tta_method       ICR Insert   None Paraphrase Substitute Translate    ICR   
mean_accuracy  27.99  24.85  25.86      27.55      24.73     26.05   90.2   

task                                                         
model                                                        
distribution                                                 
tta_method    Insert   None Paraphrase Substitute Translate  
mean_accuracy  89.77  88.99      90.12       89.2      89.0

Across Seeds


{3:                                              mean_accuracy
 task          model distribution tta_method               
 AgNewsTweets  BERT  Tweets       ICR                 89.66
                                  Insert              88.94
                                  None                88.57
                                  Paraphrase          89.03
                                  Substitute          88.67
 ...                                                    ...
 BOSS_Toxicity T5    Toxigen      Insert              65.71
                                  None                65.71
                                  Paraphrase          66.35
                                  Substitute          62.85
                                  Translate           66.99
 
 [126 rows x 1 columns],
 17:                                              mean_accuracy
 task          model distribution tta_method               
 AgNewsTweets  BERT  Tweets       ICR                 89.73
      

In [52]:
# create dataframe where there is a column for dataset, model, id_accuracy_delta, and mean_accuracy_delta
records = []
num_seeds = len(multiseed_detailed_results.keys())
print(f"Aggregating results across {num_seeds} seeds")

for seed in multiseed_detailed_results:
    for task_name in tqdm(multiseed_split_data[seed]):
        for model_name in multiseed_split_data[seed][task_name]:
            for tta_method in multiseed_split_data[seed][task_name][model_name]:
                current_tta_result = multiseed_split_data[seed][task_name][model_name][tta_method]
                records.append({
                    "dataset": task_name,
                    "model": model_name,
                    "tta_method": tta_method,
                    "ood_mean_accuracy": current_tta_result["mean_accuracy"] * 100,
                })
                records.append({
                    "dataset": task_name,
                    "model": model_name,
                    "tta_method": "None",
                    "ood_mean_accuracy": current_tta_result["mean_baseline_accuracy"] * 100,
                })

main_results_frame = pd.DataFrame(records)
for model in split_data[task_name]:
    display(model)
    display(main_results_frame[main_results_frame["model"] == model].drop(columns="model").groupby(["dataset", "tta_method"]).agg(['mean', 'std']).round(2).T)

aggregated_main_results = main_results_frame.groupby(["dataset", "model", "tta_method"]).agg(['mean', 'std']).round(2).T

Aggregating results across 4 seeds


100%|██████████| 3/3 [00:00<00:00, 21583.04it/s]


'BERT'

dataset                AgNewsTweets                                      \
tta_method                      ICR Insert   None Paraphrase Substitute   
ood_mean_accuracy mean        89.72  88.86  88.57      88.97      88.77   
                  std          0.06   0.08   0.00       0.04       0.08   

dataset                          BOSS_Sentiment                           \
tta_method             Translate            ICR Insert   None Paraphrase   
ood_mean_accuracy mean      88.5          57.17  51.66  52.05      55.24   
                  std        0.0           0.16   0.49   0.01       0.18   

dataset                                     BOSS_Toxicity                \
tta_method             Substitute Translate           ICR Insert   None   
ood_mean_accuracy mean      51.44     52.40         61.06  53.11  53.88   
                  std        0.32      0.02          0.20   0.40   0.00   

dataset                                                 
tta_method             Paraphrase Substitute Translate  
ood_mean_accuracy mean      64.67      52.70     62.08  
                  std        0.30       0.35      0.00

'T5'

dataset                AgNewsTweets                                      \
tta_method                      ICR Insert   None Paraphrase Substitute   
ood_mean_accuracy mean        90.30  89.69  88.99      89.98      89.08   
                  std          0.15   0.12   0.00       0.13       0.10   

dataset                          BOSS_Sentiment                           \
tta_method             Translate            ICR Insert   None Paraphrase   
ood_mean_accuracy mean      89.0          59.75  56.11  58.09      59.01   
                  std        0.0           0.18   0.18   0.01       0.27   

dataset                                     BOSS_Toxicity                \
tta_method             Substitute Translate           ICR Insert   None   
ood_mean_accuracy mean      54.15     56.71         63.83  57.59  58.84   
                  std        0.50      0.00          0.26   0.27   0.00   

dataset                                                 
tta_method             Paraphrase Substitute Translate  
ood_mean_accuracy mean       67.2      56.99      63.4  
                  std         0.5       0.49       0.0

'Falcon'

dataset                AgNewsTweets                                      \
tta_method                      ICR Insert   None Paraphrase Substitute   
ood_mean_accuracy mean        27.82  24.98  25.86      27.50      24.78   
                  std          0.16   0.11   0.00       0.17       0.05   

dataset                          BOSS_Sentiment                           \
tta_method             Translate            ICR Insert   None Paraphrase   
ood_mean_accuracy mean     26.05          43.79  45.82  46.77      45.15   
                  std       0.01           0.11   0.27   0.00       0.17   

dataset                                     BOSS_Toxicity                \
tta_method             Substitute Translate           ICR Insert   None   
ood_mean_accuracy mean      43.56     44.06         58.11  57.33  59.11   
                  std        0.31      0.00          0.13   0.11   0.00   

dataset                                                 
tta_method             Paraphrase Substitute Translate  
ood_mean_accuracy mean      58.90      58.87     59.25  
                  std        0.09       0.10      0.00

### Get Max STD

In [53]:
# aggregated_main_results.groupby(["dataset", "tta_method"]).drop(columns="model").agg(["std"]).groupby("tta_method").mean().round(4).T

In [54]:
# aggregated_main_results.groupby(["dataset", "model", "tta_method"]).agg(["std"]).max().round(2).T

### Build Main OOD Results LaTEX Table

In [55]:
aggregated_main_results["AgNewsTweets"]["BERT"]

tta_method                ICR  Insert   None  Paraphrase  Substitute  \
ood_mean_accuracy mean  89.72   88.86  88.57       88.97       88.77   
                  std    0.06    0.08   0.00        0.04        0.08   

tta_method              Translate  
ood_mean_accuracy mean       88.5  
                  std         0.0

In [56]:

task_order = ["BOSS_Sentiment", "BOSS_Toxicity", "AgNewsTweets"]
model_order = ["BERT", "T5", "Falcon"]
tta_order = ["None", "Insert", "Substitute", "Translate", "Paraphrase", "ICR"]
citation_map = {
    "None": "None",
    "Insert": "Insert \citep{Lu2022ImprovedTC}",
    "Substitute": "Substitute \citep{Lu2022ImprovedTC}",
    "Translate": "Translate \citep{Sennrich2015ImprovingNM}",
    "Paraphrase": "LLM--TTA: Paraphrase",
    "ICR": "LLM--TTA: ICR",
}

table_latex_lines = []
for tta_method in tta_order:
    row_entries = [citation_map[tta_method]]
    for task in task_order:
        for model in model_order:
            mean_accuracy = aggregated_main_results[task][model][tta_method]["ood_mean_accuracy"]["mean"]
            formatted_acc  = f"{mean_accuracy:.2f}\\%"
            best_acc = max([aggregated_main_results[task][model][method_name]["ood_mean_accuracy"]["mean"] for method_name in tta_order])
            is_best_method = mean_accuracy == best_acc
            if is_best_method:
                formatted_acc = "\\textbf{" + formatted_acc + "}"
            
            row_entries.append(formatted_acc)

    row_latex = "    " + " & ".join(row_entries) + " \\\\"
    print(row_latex)

    table_latex_lines.append(row_latex)
    if tta_method in ["None", "Translate"]:
        dash_line = "    " + "\cdashlinelr{1-10}"
        table_latex_lines.append(dash_line)
        print(dash_line)
            

    None & 52.05\% & 58.09\% & \textbf{46.77\%} & 53.88\% & 58.84\% & 59.11\% & 88.57\% & 88.99\% & 25.86\% \\
    \cdashlinelr{1-10}
    Insert \citep{Lu2022ImprovedTC} & 51.66\% & 56.11\% & 45.82\% & 53.11\% & 57.59\% & 57.33\% & 88.86\% & 89.69\% & 24.98\% \\
    Substitute \citep{Lu2022ImprovedTC} & 51.44\% & 54.15\% & 43.56\% & 52.70\% & 56.99\% & 58.87\% & 88.77\% & 89.08\% & 24.78\% \\
    Translate \citep{Sennrich2015ImprovingNM} & 52.40\% & 56.71\% & 44.06\% & 62.08\% & 63.40\% & \textbf{59.25\%} & 88.50\% & 89.00\% & 26.05\% \\
    \cdashlinelr{1-10}
    LLM--TTA: Paraphrase & 55.24\% & 59.01\% & 45.15\% & \textbf{64.67\%} & \textbf{67.20\%} & 58.90\% & 88.97\% & 89.98\% & 27.50\% \\
    LLM--TTA: ICR & \textbf{57.17\%} & \textbf{59.75\%} & 43.79\% & 61.06\% & 63.83\% & 58.11\% & \textbf{89.72\%} & \textbf{90.30\%} & \textbf{27.82\%} \\


In [57]:

task_order = ["BOSS_Sentiment", "BOSS_Toxicity", "AgNewsTweets"]
model_order = ["BERT", "T5", "Falcon"]
tta_order = ["None", "Insert", "Substitute", "Translate", "Paraphrase", "ICR"]
citation_map = {
    "None": "None",
    "Insert": "Insert \citep{Lu2022ImprovedTC}",
    "Substitute": "Substitute \citep{Lu2022ImprovedTC}",
    "Translate": "Translate \citep{Sennrich2015ImprovingNM}",
    "Paraphrase": "LLM--TTA: Paraphrase",
    "ICR": "LLM--TTA: ICR",
}

table_latex_lines = []
for tta_method in tta_order:
    row_entries = [citation_map[tta_method]]
    for task in task_order:
        for model in model_order:
            mean_accuracy = aggregated_main_results[task][model][tta_method]["ood_mean_accuracy"]["mean"]
            formatted_acc  = f"{mean_accuracy:.2f}\\%"
            best_acc = max([aggregated_main_results[task][model][method_name]["ood_mean_accuracy"]["mean"] for method_name in tta_order])
            is_best_method = mean_accuracy == best_acc
            if is_best_method:
                formatted_acc = "\\textbf{" + formatted_acc + "}"
            
            row_entries.append(formatted_acc)

    row_latex = "    " + " & ".join(row_entries) + " \\\\"
    print(row_latex)

    table_latex_lines.append(row_latex)
    if tta_method in ["None", "Translate"]:
        dash_line = "    " + "\cdashlinelr{1-10}"
        table_latex_lines.append(dash_line)
        print(dash_line)
            

    None & 52.05\% & 58.09\% & \textbf{46.77\%} & 53.88\% & 58.84\% & 59.11\% & 88.57\% & 88.99\% & 25.86\% \\
    \cdashlinelr{1-10}
    Insert \citep{Lu2022ImprovedTC} & 51.66\% & 56.11\% & 45.82\% & 53.11\% & 57.59\% & 57.33\% & 88.86\% & 89.69\% & 24.98\% \\
    Substitute \citep{Lu2022ImprovedTC} & 51.44\% & 54.15\% & 43.56\% & 52.70\% & 56.99\% & 58.87\% & 88.77\% & 89.08\% & 24.78\% \\
    Translate \citep{Sennrich2015ImprovingNM} & 52.40\% & 56.71\% & 44.06\% & 62.08\% & 63.40\% & \textbf{59.25\%} & 88.50\% & 89.00\% & 26.05\% \\
    \cdashlinelr{1-10}
    LLM--TTA: Paraphrase & 55.24\% & 59.01\% & 45.15\% & \textbf{64.67\%} & \textbf{67.20\%} & 58.90\% & 88.97\% & 89.98\% & 27.50\% \\
    LLM--TTA: ICR & \textbf{57.17\%} & \textbf{59.75\%} & 43.79\% & 61.06\% & 63.83\% & 58.11\% & \textbf{89.72\%} & \textbf{90.30\%} & \textbf{27.82\%} \\


## Dataset Stats

In [58]:
split_label_value_counts = {}

for split_name in multiseed_inference_logs[3]:
    if "BERT" not in split_name:
        continue

    set_name = split_name.split("_BERT")[0]
    if set_name in split_label_value_counts:
        continue

    split_frame = multiseed_inference_logs[3][split_name].to_pandas()
    label_counts = split_frame["label"].value_counts().to_dict()
    label_counts[-1] = len(split_frame)
    label_counts = {k: label_counts[k] for k in sorted(label_counts)}
    split_label_value_counts[set_name] = label_counts

display(pd.DataFrame(split_label_value_counts))

,BOSS_Sentiment_SST5,BOSS_Sentiment_Ablate_Data_SST5,BOSS_Sentiment_SemEval,BOSS_Sentiment_Ablate_Data_SemEval,BOSS_Sentiment_Dynasent,BOSS_Sentiment_Ablate_Data_Dynasent,BOSS_Toxicity_Toxigen,BOSS_Toxicity_Ablate_Data_Toxigen,BOSS_Toxicity_AdvCivil,BOSS_Toxicity_Ablate_Data_AdvCivil,BOSS_Toxicity_ImplicitHate,BOSS_Toxicity_Ablate_Data_ImplicitHate,AgNewsTweets_Tweets,AgNewsTweets_Ablate_Data_Tweets
-1,1068.0,1068.0,20622.0,20622.0,4320.0,4320.0,942.0,942.0,828.0,828.0,21480.0,21480.0,7602,7602
0,280.0,280.0,3229.0,3229.0,1440.0,1440.0,542.0,542.0,152.0,152.0,13291.0,13291.0,1902,1902
1,399.0,399.0,7059.0,7059.0,1440.0,1440.0,400.0,400.0,676.0,676.0,8189.0,8189.0,1900,1900
2,389.0,389.0,10334.0,10334.0,1440.0,1440.0,NaN,NaN,NaN,NaN,NaN,NaN,1900,1900
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900,1900


# Analyze Across Dataset Sizes

In [59]:
data_ablation_results = []
for seed in multiseed_inference_logs:
    for split_name in tqdm(multiseed_inference_logs[seed], desc=f"Seed = {seed}"):
        if "Ablate_Data" not in split_name:
            continue

        task_name = parse_task_name(split_name)
        data_count = int(split_name.split("_")[-2].replace("BERT", ""))
        tta_method = split_name.split("_")[-1]
        shift_name = split_name.split("_")[-3]
        baseline_accuracy = classification_report(multiseed_inference_logs[seed][split_name]["label"], multiseed_inference_logs[seed][split_name]["original_predicted_class"], output_dict=True)["accuracy"]
        tta_accuracy = classification_report(multiseed_inference_logs[seed][split_name]["label"], multiseed_inference_logs[seed][split_name]["tta_predicted_class"], output_dict=True)["accuracy"]
        data_ablation_results.append({
            "task": task_name,
            "shift": shift_name,
            "seed": seed,
            "data_count": data_count,
            "tta_method": formatted_method_names[tta_method],
            "baseline_accuracy": baseline_accuracy,
            "tta_accuracy": tta_accuracy,
            "acc_gain": tta_accuracy - baseline_accuracy
        })

data_ablation_results_frame = pd.DataFrame(data_ablation_results)
data_ablation_results_frame

Seed = 58: 100%|██████████| 280/280 [00:09<00:00, 28.70it/s]


,task,shift,seed,data_count,tta_method,baseline_accuracy,tta_accuracy,acc_gain
0,Sentiment,SST5,3,1500,Insert,0.682584,0.683521,0.000936
1,Sentiment,SST5,3,1500,Substitute,0.682584,0.670412,-0.012172
2,Sentiment,SST5,3,1500,Translate,0.682836,0.680037,-0.002799
3,Sentiment,SST5,3,1500,LLM-TTA: Paraphrase,0.682584,0.710674,0.028090
4,Sentiment,SST5,3,1500,LLM-TTA: ICR,0.682584,0.718165,0.035581
...,...,...,...,...,...,...,...,...
695,News,Tweets,58,4800,Insert,0.886477,0.890292,0.003815
696,News,Tweets,58,4800,Substitute,0.886477,0.890555,0.004078
697,News,Tweets,58,4800,Translate,0.886447,0.890921,0.004474
698,News,Tweets,58,4800,LLM-TTA: Paraphrase,0.886477,0.890818,0.004341


In [60]:
data_ablation_results_frame[data_ablation_results_frame["task"] == "Toxicity"]

,task,shift,seed,data_count,tta_method,baseline_accuracy,tta_accuracy,acc_gain
75,Toxicity,Toxigen,3,3000,Insert,0.610403,0.611465,0.001062
76,Toxicity,Toxigen,3,3000,Substitute,0.610403,0.614650,0.004246
77,Toxicity,Toxigen,3,3000,Translate,0.611229,0.606992,-0.004237
78,Toxicity,Toxigen,3,3000,LLM-TTA: Paraphrase,0.610403,0.590234,-0.020170
79,Toxicity,Toxigen,3,3000,LLM-TTA: ICR,0.610403,0.599788,-0.010616
...,...,...,...,...,...,...,...,...
670,Toxicity,AdvCivil,58,48000,Insert,0.336957,0.324879,-0.012077
671,Toxicity,AdvCivil,58,48000,Substitute,0.336957,0.310386,-0.026570
672,Toxicity,AdvCivil,58,48000,Translate,0.336165,0.564320,0.228155
673,Toxicity,AdvCivil,58,48000,LLM-TTA: Paraphrase,0.336957,0.609903,0.272947


In [61]:
toxic_frame = data_ablation_results_frame[data_ablation_results_frame["task"] == "Toxicity"][["task", "seed", "data_count", "tta_method", "acc_gain"]]
toxic_frame[toxic_frame["seed"] == 46]

,task,seed,data_count,tta_method,acc_gain
425,Toxicity,46,3000,Insert,0.007431
426,Toxicity,46,3000,Substitute,0.004246
427,Toxicity,46,3000,Translate,-0.004237
428,Toxicity,46,3000,LLM-TTA: Paraphrase,-0.019108
429,Toxicity,46,3000,LLM-TTA: ICR,-0.022293
...,...,...,...,...,...
495,Toxicity,46,48000,Insert,-0.010870
496,Toxicity,46,48000,Substitute,-0.026570
497,Toxicity,46,48000,Translate,0.228155
498,Toxicity,46,48000,LLM-TTA: Paraphrase,0.275362


In [62]:
# sort by seed
data_ablation_results_frame[data_ablation_results_frame["task"] == "Toxicity"].value_counts(["seed", "shift", "data_count", "tta_method"]).sort_index().to_csv("check_tox_ablations")

In [63]:
display(toxic_frame.groupby(["task", "seed", "data_count", "tta_method"]).mean("acc_gain").T["Toxicity"][3][24000])
display(toxic_frame.groupby(["task", "seed", "data_count", "tta_method"]).mean("acc_gain").T["Toxicity"][17][24000])
display(toxic_frame.groupby(["task", "seed", "data_count", "tta_method"]).mean("acc_gain").T["Toxicity"][46][24000])
display(toxic_frame.groupby(["task", "seed", "data_count", "tta_method"]).mean("acc_gain").T["Toxicity"][58][24000])

tta_method,Insert,LLM-TTA: ICR,LLM-TTA: Paraphrase,Substitute,Translate
acc_gain,-0.00274,0.051408,0.082222,-0.000073,0.058732


tta_method,Insert,LLM-TTA: ICR,LLM-TTA: Paraphrase,Substitute,Translate
acc_gain,-0.004453,0.058882,0.078594,-0.000904,0.058732


tta_method,Insert,LLM-TTA: ICR,LLM-TTA: Paraphrase,Substitute,Translate
acc_gain,-0.005399,0.056558,0.081174,0.000186,0.058732


tta_method,Insert,LLM-TTA: ICR,LLM-TTA: Paraphrase,Substitute,Translate
acc_gain,-0.003391,0.052734,0.079753,0.004563,0.058732


In [64]:
fig, axes = plt.subplots(1, 3, figsize=(3 * FIG_SIZE, FIG_SIZE - 1))
for i, task_name in enumerate(["Sentiment", "Toxicity", "News"]):
    # df = pd.concat([pd.DataFrame(pandas_form[task_name][data_count]) for data_count in pandas_form[task_name]])
    data_ablation_plotting_df = data_ablation_results_frame[data_ablation_results_frame["task"] == task_name]
    data_ablation_plotting_df = data_ablation_plotting_df[data_ablation_plotting_df["seed"] != 58]
    agg_data_ablation_plotting_df = data_ablation_plotting_df.groupby(["task", "seed", "data_count", "tta_method"]).mean("acc_gain")
    agg_data_ablation_plotting_df = agg_data_ablation_plotting_df.reset_index().sort_values(by=["tta_method"], key=lambda x: x.map(TTA_METHOD_ORDER))
    sns.lineplot(
        data=agg_data_ablation_plotting_df,
        x="data_count",
        y="acc_gain",
        hue="tta_method",
        ax=axes[i],
        linewidth=LINE_WIDTH,
        markersize=MARKER_SIZE,
        style="tta_method",
        dashes=False,
        markers=True,
        errorbar="sd",
    )

    # set x label to Training Set Size
    axes[i].set_xlabel("Training Set Size")

    # set y label to Mean Absolute Accuracy Delta
    axes[i].set_ylabel("Accuracy Gain")

    # make the y axis percents that go to the hundreds place
    axes[i].yaxis.set_major_formatter(lambda x, pos: f"{x:.0%}")

    # standardize the y axis between the min and max of df
    axes[i].set_ylim(data_ablation_plotting_df["acc_gain"].min() - 0.005, data_ablation_plotting_df["acc_gain"].max() + 0.005)

    axes[i].set_title(task_name, fontsize=TITLE_FONT_SIZE)

    # se legend to the bottom left
    axes[i].legend(loc="lower right")

    # make x axis log
    axes[i].set_xscale("log")

    # show more x axis ticks at 10%, 20%, 40%, 80%
    x_ticks = sorted(data_ablation_plotting_df["data_count"].unique().tolist())
    tick_labels = [str(int(x)) for x in x_ticks]
    axes[i].set_xticks(x_ticks, tick_labels, minor=False)

    # remove extra ticks not in x_ticks
    axes[i].set_xticks(x_ticks, minor=True)

    # rotate x axis labels
    axes[i].tick_params(axis='x', rotation=X_LABEL_ROTATION)

    # if news add addiitonal precision to the y axis
    # if task_name == "AgNewsTweets":
    axes[i].yaxis.set_major_formatter(lambda x, pos: f"{x:.1%}")

    # remove leegnd in not middle plot
    if i != 1:
        axes[i].get_legend().remove()
    else:
        # center below plot with no frame
        axes[i].legend(loc="upper center", bbox_to_anchor=(0.5, -0.30), ncol=5, frameon=False, fontsize=14)

# add padding for labels
fig.subplots_adjust(wspace=WSPACE + 0.05, hspace=WSPACE)

if not os.path.exists("figures_tmlr/"):
    os.makedirs("figures_tmlr/")

fig.savefig("figures_tmlr/method_analysis_data_ablation.pdf", bbox_inches="tight")
fig.savefig("figures_tmlr/method_analysis_data_ablation.png", bbox_inches="tight")

RuntimeError: Failed to process string with tex because latex could not be found

Error in callback <function _draw_all_if_interactive at 0x14f6fa793760> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1200x300 with 3 Axes>

## Aggregation Ablation Study

In [ ]:
def aggregate_predictions(predictions, num_predictions, use_test_input):
    try:
        ablation_preds = predictions[:num_predictions]
        if use_test_input:
            ablation_preds = ablation_preds.tolist() + [predictions[-1]]
        
        mean_distribution = np.mean(ablation_preds, axis=0) if len(ablation_preds) > 1 else ablation_preds[0]
        predicted_class = np.argmax(mean_distribution)
        return predicted_class
    except:
        return -1

perf_records = []
for seed in multiseed_inference_logs:
    agg_ablation_splits = [name for name in multiseed_inference_logs[seed] if "Ablate" not in name and "BERT" in name]
    for split_name in tqdm(agg_ablation_splits, desc=f"Seed = {seed}"):
        dataset = parse_task_name(split_name)
        distribution = parse_distribution(split_name)
        model = parse_model(split_name)
        method = parse_tta_method(split_name)

        current_frame = multiseed_inference_logs[seed][split_name].to_pandas()
        for use_source in [False, True]:
            for num_augmentations in range(1, 5):
                judgments = current_frame["tta_all_class_probs"].apply(lambda x: aggregate_predictions(x, num_augmentations, use_source))
                accuracy = classification_report(current_frame["label"], judgments, output_dict=True, zero_division=0.0)["accuracy"]
                perf_records.append({
                    "dataset": dataset,
                    "seed": seed,
                    "distribution": distribution,
                    "model": model,
                    "method": formatted_method_names[method],
                    "use_source": use_source,
                    "num_augmentations": num_augmentations,
                    "accuracy": accuracy,
                    "delta": accuracy - no_tta_accuracies[seed][split_name],
                })

aggregation_ablation_frame = pd.DataFrame(perf_records)
aggregation_ablation_frame

In [ ]:
test_frame = aggregation_ablation_frame.copy()[["dataset", "model", "method", "num_augmentations", "use_source", "delta"]]
test_frame = test_frame[test_frame["use_source"] == True]
test_frame.drop(columns="use_source", inplace=True)
test_frame = test_frame.groupby(["dataset", "model", "method", "num_augmentations"]).agg(["mean", "std"]).T * 100
test_frame

In [ ]:
# select where distribution != ID
aggregation_ablation_frame = aggregation_ablation_frame[aggregation_ablation_frame["distribution"] != "ID"]
fig, axes = plt.subplots(1, 3, figsize=(3 * FIG_SIZE, FIG_SIZE - 1))
for index, task_name in enumerate(["Sentiment", "Toxicity", "News"]):
    # get the current task frame
    current_frame = aggregation_ablation_frame[aggregation_ablation_frame["dataset"] == task_name]
    current_frame = current_frame[current_frame["use_source"] == True]
    current_frame = current_frame.drop(columns="use_source")
    current_frame = current_frame.sort_values(by=["method"], key=lambda x: x.map(TTA_METHOD_ORDER))
    current_frame = aggregation_ablation_frame[aggregation_ablation_frame["dataset"] == task_name]
    current_frame = current_frame.groupby(["dataset", "seed", "num_augmentations", "method"]).mean("delta")
    current_frame = current_frame.reset_index().sort_values(by=["method"], key=lambda x: x.map(TTA_METHOD_ORDER))
    
    sns.lineplot(
        data=current_frame,
        x="num_augmentations",
        y="delta",
        hue="method",
        style="method",
        ax=axes[index],
        linewidth=LINE_WIDTH,
        dashes=False,
        markers=True,
        markersize=MARKER_SIZE,
        errorbar="sd",
    )

    # set x label to Number of Augmentations
    axes[index].set_xlabel("Number of Augmentations")

    # set y label to Accuracy
    axes[index].set_ylabel("Accuracy Gain")

    # make the y axis percents that go to the hundreds place
    axes[index].yaxis.set_major_formatter(lambda x, pos: f"{x:.0%}")

    # y range is -1 and +1 the min and max of the data
    axes[index].set_ylim(current_frame["delta"].min() - 0.005, current_frame["delta"].max() + 0.005)

    # set x ticks as 1, 2, 3, 4
    axes[index].set_xticks(range(1, 5))

    # set title to Sentiment, Toxicity, or News
    axes[index].set_title(task_name, fontsize=TITLE_FONT_SIZE)

    # se legend to the bottom left
    axes[index].legend(loc="lower right")

    # remove leegnd in not middle plot
    if index != 1:
        axes[index].get_legend().remove()
    else:
        # center below plot with no frame
        axes[index].legend(loc="upper center", bbox_to_anchor=(0.5, -0.20), ncol=5, frameon=False, fontsize=14)
    
    # have y ticks to the tenths place
    axes[index].yaxis.set_major_formatter(lambda x, pos: f"{x:.1%}")

# add padding for labels
fig.subplots_adjust(wspace=WSPACE + 0.05, hspace=WSPACE)

fig.savefig("figures_tmlr/method_analysis_aggrgeation_ablation.pdf", bbox_inches="tight")
fig.savefig("figures_tmlr/method_analysis_aggrgeation_ablation.png", bbox_inches="tight")

## Does TTA Affect Some Classes More Than Others?

In [ ]:
ood_sentiment_icr_data = None
ood_toxicity_icr_data = None
ood_tweets_icr_data = None

for seed in multiseed_inference_logs:
    for split_name in tqdm(multiseed_inference_logs[seed], desc=f"Seed = {3}"):
        if "ICR" not in split_name and "ID" in split_name or "BERT" not in split_name or "Ablate" in split_name:
            continue

        current_frame = multiseed_inference_logs[seed][split_name].to_pandas()
        current_frame["seed"] = seed
        current_frame["dataset"] = parse_distribution(split_name)
        current_frame["outcome"] = current_frame["outcome"].apply(lambda o: "Unfixed Mistake" if o == "NA" else o)

        if "Sentiment" in split_name:
            if ood_sentiment_icr_data is None:
                ood_sentiment_icr_data = current_frame
            else:
                ood_sentiment_icr_data = pd.concat([ood_sentiment_icr_data, current_frame])

        if "Toxicity" in split_name:
            if ood_toxicity_icr_data is None:
                ood_toxicity_icr_data = current_frame
            else:
                ood_toxicity_icr_data = pd.concat([ood_toxicity_icr_data, current_frame])

        if "Tweets" in split_name:
            if ood_tweets_icr_data is None:
                ood_tweets_icr_data = current_frame
            else:
                ood_tweets_icr_data = pd.concat([ood_tweets_icr_data, current_frame])


display(ood_sentiment_icr_data.value_counts("dataset"))
display(ood_toxicity_icr_data.value_counts("dataset"))
display(ood_tweets_icr_data.value_counts("dataset"))
assert len(ood_sentiment_icr_data.value_counts("dataset")) == 3
assert len(ood_toxicity_icr_data.value_counts("dataset")) == 3
assert len(ood_tweets_icr_data.value_counts("dataset")) == 1

In [ ]:
outcome_ordering = {
    "New Correct": 0,
    "New Mistake": 1,
}
sentiment_labels = {
    0: "Negative",
    1: "Positive",
    2: "Neutral",
}
sentiment_label_order = dict(zip(sentiment_labels.values(), range(len(sentiment_labels))))
multiseed_sentiment_icr_outcomes_percents_frame = None
for seed in multiseed_inference_logs:
    seed_ood_sentiment_icr_data = ood_sentiment_icr_data[ood_sentiment_icr_data["seed"] == seed]
    seed_ood_sentiment_icr_data["label"] = seed_ood_sentiment_icr_data["label"].apply(lambda l: sentiment_labels[l])
    outcome_percents = seed_ood_sentiment_icr_data.value_counts(["label", "outcome"], normalize=True).reset_index()
    outcome_percents = outcome_percents[outcome_percents["outcome"].str.contains("New")]
    outcome_percents["seed"] = seed

    if multiseed_sentiment_icr_outcomes_percents_frame is None:
        multiseed_sentiment_icr_outcomes_percents_frame = outcome_percents
    else:
        multiseed_sentiment_icr_outcomes_percents_frame = pd.concat([multiseed_sentiment_icr_outcomes_percents_frame, outcome_percents])
    
    multiseed_sentiment_icr_outcomes_percents_frame.sort_values(by=["label"], key=lambda x: x.map(toxicity_label_order), inplace=True)
    multiseed_sentiment_icr_outcomes_percents_frame.sort_values(by=["outcome"], key=lambda x: x.map(outcome_ordering), inplace=True)

print("Sentiment")
display(multiseed_sentiment_icr_outcomes_percents_frame.T)

toxicity_labels = {
    0: "Non-Toxic",
    1: "Toxic",
}
toxicity_label_order = dict(zip(toxicity_labels.values(), range(len(toxicity_labels))))
multiseed_toxicity_icr_outcomes_percents_frame = None
for seed in multiseed_inference_logs:
    seed_ood_toxicity_icr_data = ood_toxicity_icr_data[ood_toxicity_icr_data["seed"] == seed]
    seed_ood_toxicity_icr_data["label"] = seed_ood_toxicity_icr_data["label"].apply(lambda l: toxicity_labels[l])
    outcome_percents = seed_ood_toxicity_icr_data.value_counts(["label", "outcome"], normalize=True).reset_index()
    outcome_percents = outcome_percents[outcome_percents["outcome"].str.contains("New")]
    outcome_percents["seed"] = seed

    if multiseed_toxicity_icr_outcomes_percents_frame is None:
        multiseed_toxicity_icr_outcomes_percents_frame = outcome_percents
    else:
        multiseed_toxicity_icr_outcomes_percents_frame = pd.concat([multiseed_toxicity_icr_outcomes_percents_frame, outcome_percents])

    multiseed_toxicity_icr_outcomes_percents_frame.sort_values(by=["label"], key=lambda x: x.map(toxicity_label_order), inplace=True)

print("Toxicity")
display(multiseed_toxicity_icr_outcomes_percents_frame.T)

agt_labels = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech",
}
agt_label_order = dict(zip(agt_labels.values(), range(len(agt_labels))))
multiseed_tweets_icr_outcomes_percents_frame = None
for seed in multiseed_inference_logs:
    seed_ood_tweets_icr_data = ood_tweets_icr_data[ood_tweets_icr_data["seed"] == seed]
    seed_ood_tweets_icr_data["label"] = seed_ood_tweets_icr_data["label"].apply(lambda l: agt_labels[l])
    outcome_percents = seed_ood_tweets_icr_data.value_counts(["label", "outcome"], normalize=True).reset_index()
    outcome_percents = outcome_percents[outcome_percents["outcome"].str.contains("New")]
    outcome_percents["seed"] = seed

    if multiseed_tweets_icr_outcomes_percents_frame is None:
        multiseed_tweets_icr_outcomes_percents_frame = outcome_percents
    else:
        multiseed_tweets_icr_outcomes_percents_frame = pd.concat([multiseed_tweets_icr_outcomes_percents_frame, outcome_percents])

    multiseed_tweets_icr_outcomes_percents_frame.sort_values(by=["label"], key=lambda x: x.map(agt_label_order), inplace=True)
    

print("News")
display(multiseed_tweets_icr_outcomes_percents_frame.T)


In [ ]:
plt.clf()
fig, axes = plt.subplots(1, 3, figsize=(3 * FIG_SIZE, FIG_SIZE - 1))

sns.barplot(
    data=multiseed_sentiment_icr_outcomes_percents_frame,
    ax=axes[0],
    x="label",
    y="proportion",
    hue="outcome",
)

sns.barplot(
    data=multiseed_toxicity_icr_outcomes_percents_frame,
    ax=axes[1],
    x="label",
    y="proportion",
    hue="outcome",
)

sns.barplot(
    data=multiseed_tweets_icr_outcomes_percents_frame,
    ax=axes[2],
    x="label",
    y="proportion",
    hue="outcome",
)

for i in range(3):
    # make y axis percents
    axes[i].yaxis.set_major_formatter(lambda x, pos: f"{x:.0%}")

    # standardize between 0 and 0.1
    # axes[i].set_ylim(0, 0.09)

    # set y label to percent of overall outcomes
    axes[i].set_ylabel("Percent of Outcomes")

    # remove x label
    axes[i].set_xlabel("")

    # make the y axis percents that go to the hundreds place
    axes[index].yaxis.set_major_formatter(lambda x, pos: f"{x:.1%}")

    if i == 2:
        # rotate the x labels
        axes[i].tick_params(axis="x", rotation=X_LABEL_ROTATION)

    # have fewer ticks on the y axis
    axes[i].locator_params(axis="y", nbins=5)

    # set titles
    title_text = {
        0: "Sentiment",
        1: "Toxicity",
        2: "News",
    }
    axes[i].set_title(title_text[i], fontsize=TITLE_FONT_SIZE)

    # have a single legend which is centered below the plot
    if i == 1:
        axes[i].legend(loc="upper center", bbox_to_anchor=(0.5, -0.20), ncol=3, frameon=False, fontsize=14)
    else:
        axes[i].get_legend().remove()

# add more horizental spacing for y labels
fig.subplots_adjust(wspace=WSPACE + 0.05, hspace=WSPACE)
fig.savefig("figures_tmlr/method_analysis_class_analysis.pdf", bbox_inches="tight")
fig.savefig("figures_tmlr/method_analysis_class_analysis.png", bbox_inches="tight")
fig.show()

# Entropy-Based Selective Augmentation 

In [ ]:
def should_augment_entropy(threshold, row):
    return row["original_prediction_entropy"] >= threshold


def get_entropy_threshold_accuracy(threshold, inference_logs_frame):
    threshold_judgments = inference_logs_frame.apply(lambda row: row["tta_predicted_class"] if should_augment_entropy(threshold, row) else row["original_predicted_class"], axis=1)
    report = classification_report(inference_logs_frame["label"], threshold_judgments, digits=4, output_dict=True, zero_division=0)
    llm_call_count = inference_logs_frame.apply(lambda row: should_augment_entropy(threshold, row), axis=1).sum()
    llm_call_rate = llm_call_count / len(inference_logs_frame)
    return report["accuracy"], llm_call_rate


def should_augment_softmax(threshold, row):
    try:
        return row["tta_all_class_probs"][-1].max() < threshold
    except:
        return False


def get_max_softmax_threshold_accuracy(threshold, inference_logs_frame):
    threshold_judgments = inference_logs_frame.apply(lambda row: row["tta_predicted_class"] if should_augment_softmax(threshold, row) else row["original_predicted_class"], axis=1)
    report = classification_report(inference_logs_frame["label"], threshold_judgments, digits=4, output_dict=True, zero_division=0)
    llm_call_count = (inference_logs_frame["original_prediction_entropy"] >= threshold).sum()
    llm_call_rate = llm_call_count / len(inference_logs_frame)
    return report["accuracy"], llm_call_rate

thresholds = np.arange(0, 1.2, 0.0001)
print(f"Number of thresholds: {len(thresholds)}")
thresholds

## Get OOD Performance At Thresholds

In [ ]:
id_optimal_entropy_thresholds = {}
thresholds = np.arange(0, 1.2, 0.0001)
print(f"Number of thresholds: {len(thresholds)}")

if os.path.exists("data/id_optimal_entropy_thresholds.json"):
    with open("data/id_optimal_entropy_thresholds.json", "r") as f:
        id_optimal_entropy_thresholds = json.load(f)
else:
    for split in [dataset for dataset in inference_logs if not is_entropy_split(dataset) and "Ablate" not in dataset and "ID_BERT" in dataset]:
        if "Paraphrase" not in split and "ICR" not in split:
            continue

        print(split)
        best_entropy_threshold = None
        split_frame = inference_logs[split].to_pandas()
        unique_predicted_classes = [class_label for class_label in split_frame["tta_predicted_class"].unique() if class_label != -1] 
        
        threshold_performances = []
        for threshold in tqdm(thresholds):
            accuracy, llm_call_rate = get_entropy_threshold_accuracy(threshold, split_frame)
            beta = 1/500
            rate_term = 1 - llm_call_rate
            threshold_score = (1 + beta ** 2) * ((accuracy * rate_term) / ((beta ** 2) * accuracy + rate_term))
            threshold_perf = {
                "threshold": threshold,
                "accuracy": accuracy,
                "score": threshold_score,
                "llm_call_rate": f"{llm_call_rate:.2f}%",
            }
            threshold_performances.append(threshold_perf)

            # if best_entropy_threshold is None or accuracy > best_entropy_threshold["accuracy"]:
            if best_entropy_threshold is None or threshold_score > best_entropy_threshold["score"]:
                best_entropy_threshold = threshold_perf

        pd.DataFrame(threshold_performances).to_csv(f"data/threshold_performances_{split}.csv", index=False)
        id_optimal_entropy_thresholds[split] = best_entropy_threshold
        print(f"Best Entropy Threshold: {best_entropy_threshold}")

print(json.dumps(id_optimal_entropy_thresholds, indent=4))

Use the best ID test set entropy instead of the best OOD entropy

In [ ]:
def map_ood_to_id_entropy(split_name):
    method_name = split_name.split("_")[-1]
    model_name = split_name.split("_")[-2]
    distribution_name = split_name.split("_")[-3]
    task_name = split_name.replace(f"_{distribution_name}_{model_name}_{method_name}", "")
    return f"{task_name}_ID_{model_name}_{method_name}"

map_ood_to_id_entropy("BOSS_Toxicity_Toxigen_BERT_ICR")

In [ ]:
file_name = "data/id_optimal_entropy_thresholds.json"
if not os.path.exists(file_name):
    with open(file_name, "w") as f:
        json.dump(id_optimal_entropy_thresholds, f, indent=4)
else:
    print("File already exists")

In [ ]:
perf_records = []
for seed in multiseed_inference_logs:
    for split_name in tqdm(multiseed_inference_logs[seed], desc=f"Seed = {seed}"):
        is_bert_split = "BERT" in split_name
        is_ablate_frame = "Ablate" in split_name
        is_llm_tta_split = "ICR" in split_name or "Paraphrase" in split_name
        if not is_bert_split or not is_llm_tta_split or is_ablate_frame:
            continue

        split_logs = multiseed_inference_logs[seed][split_name].to_pandas()

        # perf_records.append({
        #     "split": split_name,
        #     "seed": seed,
        #     "tta": "None",
        #     "accuracy": classification_report(split_logs["label"], split_logs["original_predicted_class"], digits=4, zero_division=0, output_dict=True)["accuracy"],
        # })

        id_split_name = map_ood_to_id_entropy(split_name)
        optimal_entropy_threshold = id_optimal_entropy_thresholds[id_split_name]["threshold"]
        accuracy = get_entropy_threshold_accuracy(optimal_entropy_threshold, split_logs)[0]
        tta_method = parse_tta_method(split_name)
        task = parse_task_name(split_name)
        dataset = parse_distribution(split_name)
        perf_records.append({
            "task": task,
            "dataset": dataset,
            "seed": seed,
            "tta": tta_method,
            "accuracy": accuracy * 100,
            "augmentation_rate": split_logs.apply(lambda row: should_augment_entropy(optimal_entropy_threshold, row), axis=1).sum() / len(split_logs) * 100,
        })

perf_records

In [ ]:
np.array([56.372047, 56.169792, 56.442666, 56.550481]).std()

In [ ]:
results_frame = pd.DataFrame(perf_records).groupby(["seed", "task", "tta"]).mean(["accuracy", "augmentation_rate"]).reset_index()
display(results_frame)
results_frame.drop(columns=["seed"]).groupby(["task", "tta"]).agg(["mean", "std"]).round(2).T

In [ ]:
results_frame = pd.DataFrame(perf_records)
display(results_frame.drop(columns="dataset").groupby(["seed", "task", "tta"]).agg(["mean", "std"]).T)

aggregated_esa_results = results_frame.drop(columns=["dataset", "seed", "augmentation_rate"]).groupby(["task", "tta"]).agg(["mean", "std"]).T
display(aggregated_esa_results)



In [ ]:
results_frame = pd.DataFrame(perf_records)
results_frame["Dataset"] = results_frame["split"].apply(lambda s: s.split("_")[-4])
results_frame["Distribution"] = results_frame["split"].apply(lambda s: s.split("_")[-3])
results_frame["Model"] = results_frame["split"].apply(lambda s: s.split("_")[-2])
results_frame["TTA Method"] = results_frame["split"].apply(lambda s: s.split("_")[-1])
# results_frame["Accuracy"] = results_frame["accuracy"]
results_frame["Baseline Delta"] = results_frame.apply(lambda row: row["accuracy"] - results_frame[(results_frame["split"] == row["split"]) & (results_frame["tta"] == "None")]["accuracy"].values[0], axis=1)
results_frame.drop(columns=["split"], inplace=True)
results_frame.rename(columns={"tta": "Selective Method", "accuracy": "Accuracy", "augmentation_rate": "Augmentation Rate"}, inplace=True)

aggregated_results = results_frame.groupby(["Dataset", "Distribution", "Model", "TTA Method", "Selective Method"]).mean().round(4) * 100
aggregated_results = aggregated_results.sort_values(by=["Dataset", "Distribution", "Model", "TTA Method", "Accuracy"], ascending=False)

print("Overall Results")
# display(aggregated_results)

# Average each TTA Method and Selective Method over distributions
results_frame["ID"] = results_frame["Distribution"].apply(lambda d: "ID" in d)
results_frame.drop(columns=["Distribution"], inplace=True)
results_frame = results_frame[["Dataset", "ID", "Model", "TTA Method", "Selective Method", "Accuracy", "Baseline Delta", "Augmentation Rate"]]
aggregated_results = results_frame.groupby(["Dataset", "ID", "Model", "TTA Method", "Selective Method"]).mean().round(4) * 100
aggregated_results = aggregated_results.sort_values(by=["Dataset", "ID", "Model", "TTA Method", "Baseline Delta", "Accuracy"], ascending=False)
print("Average Results")
for dataset in ["Sentiment", "Toxicity", "AgNewsTweets"]:
    for tta_method in ["Paraphrase", "ICR"]:
    # for tta_method in ["ICR"]:
        print(f"Dataset: {dataset}, TTA Method: {tta_method}")
        # display(aggregated_results.loc[dataset, :, :, tta_method, :])
        # only show entropy-based
        display(aggregated_results.loc[dataset, :, :, tta_method, "entropy-based"])

## ICR Embeddings Analysis

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_built() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-large").to(device).eval()

In [ ]:
def get_mean_aug_embedding(augmentations):
    with torch.no_grad():
        embeddings = []
        for augmentation in augmentations:
            tokenized_text = tokenizer(augmentation, return_tensors="pt", padding=True, truncation=True, max_length=512)
            input_ids = tokenized_text["input_ids"].to(device)
            outputs = model(input_ids).pooler_output[0]
            embeddings.append(outputs)

        return torch.stack(embeddings).mean(dim=0).cpu().numpy()

In [ ]:
icr_sst_logs = inference_logs["BOSS_Sentiment_SST5_BERT_ICR"].to_pandas().sample(1072, random_state=RANDOM_SEED)
icr_sst_logs.head()

In [ ]:
icr_amazon_logs = inference_logs["BOSS_Sentiment_ID_BERT_ICR"].to_pandas().sample(1072, random_state=RANDOM_SEED)
icr_amazon_logs["orig_embedding"] = icr_amazon_logs.progress_apply(lambda row: get_mean_aug_embedding([row["original_text"]]), axis=1)
icr_amazon_logs["Distribution"] = "In-Distribution"
icr_amazon_logs.head()

In [ ]:
icr_sst_logs["orig_embedding"] = icr_sst_logs.progress_apply(lambda row: get_mean_aug_embedding([row["original_text"]]), axis=1)
icr_sst_logs["augs_embedding"] = icr_sst_logs.progress_apply(lambda row: get_mean_aug_embedding(row["augmentations"]), axis=1)
icr_sst_logs["Distribution"] = "Out-of-Distribution"
icr_sst_logs.head()

In [ ]:
id_embeddings_frame = icr_amazon_logs[["Distribution", "orig_embedding"]].rename(columns={"orig_embedding": "Embedding"})
ood_embeddings_frame = icr_sst_logs[["Distribution", "orig_embedding"]].rename(columns={"orig_embedding": "Embedding"})
ood_augs_embeddings_frame = icr_sst_logs[["Distribution", "augs_embedding"]].rename(columns={"augs_embedding": "Embedding"})
ood_augs_embeddings_frame["Distribution"] = "Out-of-Distribution Augmented"
all_embeddings_frame = pd.concat([id_embeddings_frame, ood_embeddings_frame, ood_augs_embeddings_frame])
all_embeddings_frame.head()

In [ ]:
id_embeddings_frame_centroid = torch.Tensor(id_embeddings_frame["Embedding"].mean())
ood_embeddings_frame_centroid = torch.Tensor(ood_embeddings_frame["Embedding"].mean())
ood_augs_embeddings_frame_centroid = torch.Tensor(ood_augs_embeddings_frame["Embedding"].mean())

cos = CosineSimilarity(dim=0, eps=1e-6)
print(f"OOD Original vs ID: {cos(ood_embeddings_frame_centroid, id_embeddings_frame_centroid):.4f}")
print(f"OOD Augmented vs ID: {cos(ood_augs_embeddings_frame_centroid, id_embeddings_frame_centroid):.4f}")

In [ ]:
fit_input = all_embeddings_frame["Embedding"].to_list()
umap_2d = UMAP(n_components=2, init='random', random_state=0)
all_embeddings_projections = umap_2d.fit_transform(fit_input)
all_embeddings_projections

In [ ]:
all_embeddings_frame["UMAP 1"] = all_embeddings_projections[:, 0]
all_embeddings_frame["UMAP 2"] = all_embeddings_projections[:, 1]
fig, axes = plt.subplots(1, 1, figsize=(FIG_SIZE * 1.5, FIG_SIZE * 1.5))
sns.scatterplot(data=all_embeddings_frame, x="UMAP 1", y="UMAP 2", hue="Distribution", ax=axes)

# no x and y labels
axes.set_xlabel("")
axes.set_ylabel("")

# set legend below the figure
axes.legend(loc="upper center", bbox_to_anchor=(0.5, -0.10), ncol=3, frameon=False, fontsize=14)

# no grid
axes.grid(False)

# no ticks
axes.set_xticks([])
axes.set_yticks([])

# sns.set_style("whitegrid")
fig.savefig("figures_tmlr/method_analysis_umap_embeddings.pdf", bbox_inches="tight")
fig.savefig("figures_tmlr/method_analysis_umap_embeddings.png", bbox_inches="tight")
# sns.set_style("darkgrid")

### Pairwise Cosine Similarities

In [ ]:
id_embeddings = all_embeddings_frame[all_embeddings_frame["Distribution"] == "In-Distribution"]["Embedding"].to_list()
ood_embeddings = all_embeddings_frame[all_embeddings_frame["Distribution"] == "Out-of-Distribution"]["Embedding"].to_list()
ood_augs_embeddings = all_embeddings_frame[all_embeddings_frame["Distribution"] == "Out-of-Distribution Augmented"]["Embedding"].to_list()

all_embeddings_frame["ID Distances"] = all_embeddings_frame.progress_apply(lambda row: [cos(torch.Tensor(row["Embedding"]), torch.Tensor(id_embedding)) for id_embedding in id_embeddings], axis=1)
all_embeddings_frame["OOD Distances"] = all_embeddings_frame.progress_apply(lambda row: [cos(torch.Tensor(row["Embedding"]), torch.Tensor(ood_embedding)) for ood_embedding in ood_embeddings], axis=1)
all_embeddings_frame["OOD Augmented Distances"] = all_embeddings_frame.progress_apply(lambda row: [cos(torch.Tensor(row["Embedding"]), torch.Tensor(ood_aug_embedding)) for ood_aug_embedding in ood_augs_embeddings], axis=1)

In [ ]:
all_id_distance_points = []
for distance in all_embeddings_frame[all_embeddings_frame["Distribution"] == "In-Distribution"]["ID Distances"]:
    all_id_distance_points.extend(distance)

all_ood_distance_points = []
for distance in all_embeddings_frame[all_embeddings_frame["Distribution"] == "Out-of-Distribution"]["ID Distances"]:
    all_ood_distance_points.extend(distance)

all_ood_aug_distance_points = []
for distance in all_embeddings_frame[all_embeddings_frame["Distribution"] == "Out-of-Distribution Augmented"]["ID Distances"]:
    all_ood_aug_distance_points.extend(distance)

print(f"Mean similarity (ID, ID): {np.mean(all_id_distance_points):.4f}")
print(f"Mean similarity (ID, OOD): {np.mean(all_ood_distance_points):.4f}")
print(f"Mean similarity (ID, OOD Augmented): {np.mean(all_ood_aug_distance_points):.4f}")

In [ ]:
all_embeddings_frame.head()